# Simple simulation without any actions taken

The goal of this file is to run a simple simulation with a plot of the temperatures inside and to calculate the total energy use for one month.

<br>

All the results can be found in the monitor.csv file. This file is under "examples" -> "env name" (in this case Eplus-demo-V1) -res and the new number after simulation (current is 27). Also progress.csv is available.

<br>

Questions:
-   Why does every month only last 23 days?
-   How to change the schedule. Now the temperature setpoint is 21,1C from 07:00 untill 12:00. The rest of the day it is 12,8C
    -   Want to change this to 07:00-19:00


<br>
How to deal with weekends/ non-working hours? Just set it to 12 degrees or completely off.




# Rule-based controller

In [1]:
from typing import List, Any, Sequence
from sinergym.utils.common import get_season_comfort_range
from datetime import datetime
import gym
import numpy as np
import sinergym
from sinergym.utils.wrappers import LoggerWrapper


/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [2]:
from sinergym.utils.controllers import RBC5Zone

class MyRuleBasedController(RBC5Zone):

    def act(self, observation: List[Any]) -> Sequence[Any]:
        """Select action based on outdoor air drybulb temperature and daytime.

        Args:
            observation (List[Any]): Perceived observation.

        Returns:
            Sequence[Any]: Action chosen.
        """
        obs_dict = dict(zip(self.variables['observation'], observation))

        out_temp = obs_dict['Site Outdoor Air Drybulb Temperature(Environment)']

        begin_working_hour = 7
        end_working_hour = 19

        day = int(obs_dict['day'])
        month = int(obs_dict['month'])
        hour = int(obs_dict['hour'])
        year = int(obs_dict['year'])

        summer_start_date = datetime(year, 6, 1)
        summer_final_date = datetime(year, 9, 30)

        current_dt = datetime(year, month, day)

        # Get season comfort range
        if current_dt >= summer_start_date and current_dt <= summer_final_date:
            season_comfort_range = self.setpoints_summer
        else:
            season_comfort_range = self.setpoints_winter
        season_comfort_range = get_season_comfort_range(1991,month, day)
        # Update setpoints
        in_temp = obs_dict['Zone Air Temperature(SPACE1-1)']

        current_heat_setpoint = obs_dict[
            'Zone Thermostat Heating Setpoint Temperature(SPACE1-1)']
        current_cool_setpoint = obs_dict[
            'Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)']

        new_heat_setpoint = current_heat_setpoint
        new_cool_setpoint = current_cool_setpoint

        std_heat_setpoint = new_heat_setpoint
        std_cool_setpoint = new_cool_setpoint

        # if in_temp < season_comfort_range[0]:
        #     new_heat_setpoint = current_heat_setpoint + 1
        #     new_cool_setpoint = current_cool_setpoint + 1
        # elif in_temp > season_comfort_range[1]:
        #     new_cool_setpoint = current_cool_setpoint - 1
        #     new_heat_setpoint = current_heat_setpoint - 1


        action = (new_heat_setpoint, new_cool_setpoint)

        if current_dt.weekday() >= 5 or hour not in range(begin_working_hour,end_working_hour):
            action = (18, 25.5)
        else:
            action = (20.0, 23.5)
            
        return action

In [3]:

experiment_date = datetime.today().strftime('%Y-%m-%d %H:%M')
#environment = 'Eplus-demo-v1'
environment  = "Eplus-5Zone-hot-continuous-v1"
weather_file = "USA_NY_New.York-J.F.Kennedy.Intl.AP.744860_TMY3.epw"
#weather_file = "NLD_Amsterdam.062400_IWEC.epw"

#choose the simulation period
begin_day = 1
begin_month = 1
begin_year = 2022
end_day = 1
end_month = 2
end_year = 2022

#need to change the schedule
#check for weekends



extra_params={'timesteps_per_hour' : 4,
              'runperiod' : (begin_day,begin_month,begin_year,end_day,end_month,end_year)}

env = gym.make(environment, 
                weather_file=weather_file)



# env = gym.make(
#     environment#,
#    # config_params = extra_params#,
#     # action_variables=[],
#     # action_space=gym.spaces.Box(
#     #     low=0,
#     #     high=0,
#     #     shape=(0,)),
#     # action_definition=None,)
# )

env = LoggerWrapper(env)

# create rule-based controller
agent = MyRuleBasedController(env)

for i in range(1):
    obs = env.reset()
    rewards = []
    done = False
    current_month = 0
    while not done:
        action = agent.act(obs)
       # a = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        rewards.append(reward)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', sum(rewards), info)
        if current_month == 2:
            done = True

env.close()

[2023-01-04 14:23:06,380] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2023-01-04 14:23:06,383] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-01-04 14:23:06,407] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-01-04 14:23:06,424] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2023-01-04 14:23:06,458] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Setting up action definition in building model if exists...
[2023-01-04 14:23:06,515] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...


/usr/local/lib/python3.10/dist-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


[2023-01-04 14:23:06,886] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-continuous-v1-res20/Eplus-env-sub_run1
Reward:  -0.8661416491379251 {'timestep': 1, 'time_elapsed': 900, 'year': 1991, 'month': 1, 'day': 1, 'hour': 0, 'total_power': 4543.82625749621, 'total_power_no_units': -0.454382625749621, 'comfort_penalty': -1.2779006725262292, 'abs_comfort': 1.2779006725262292, 'temperatures': [18.72209932747377], 'out_temperature': -9.049999999999999, 'action_': [18, 25.5]}
Reward:  -2415.1307950430582 {'timestep': 2976, 'time_elapsed': 2678400, 'year': 1991, 'month': 2, 'day': 1, 'hour': 0, 'total_power': 6017.331745810768, 'total_power_no_units': -0.6017331745810768, 'comfort_penalty': -2.0000014876789187, 'abs_comfort': 2.0000014876789187, 'temperatures': [17.99999851232108], 'out_temperature': -1.2, 'action_': [18, 25.5]}
[2023-01-04 14:24:30,827] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_R